In [15]:
import numpy as np
import pandas as pd
import math
import numpy as np
from numba import jit
#import numba_special
#import numba_scipy
#from numba_scipy.special import erf as erfvec
from scipy.special import erf as erfvec
import time as t
from math import log
from math import exp, sqrt, pi
from math import erf as erfmath
from numba import njit


In [ ]:
## Load Simulation Data

In [2]:
#hazard = pd.read_csv('/Users/JUSC/Documents/xgbsurv_benchmarking/implementation_testing/simulation_data/survival_simulation_preds_1000.csv').to_numpy()
hazard = log_hazard = np.random.normal(0, 1, 1000)
df = pd.read_csv('/Users/JUSC/Documents/xgbsurv_benchmarking/implementation_testing/simulation_data/survival_simulation_1000.csv')
#df.event = 1
df.sort_values(by='time', inplace=True)
df.head(2)

,x_1,x_2,x_3,x_4,x_5,time,event
104,10.782665,6.321241,0.154621,19.208570,17.674405,0.000330,1.0
638,4.839507,9.959547,0.141726,19.185224,12.585120,0.000471,1.0


In [16]:
PDF_PREFACTOR: float = 0.3989424488876037
SQRT_TWO: float = 1.4142135623730951
SQRT_EPS: float = 1.4901161193847656e-08
EPS: float = 2.220446049250313e-16
CDF_ZERO: float = 0.5

## Comparison without erf functions

In [17]:
@jit(nopython=True, cache=True, fastmath=True)
def difference_kernels(a, b, bandwidth):
    difference: np.array = np.empty(shape=(a.shape[0], b.shape[0]))
    kernel_matrix: np.array = np.empty(shape=(a.shape[0], b.shape[0]))
    integrated_kernel_matrix: np.array = np.empty(
        shape=(a.shape[0], b.shape[0])
    )
    for ix in range(a.shape[0]):
        for qx in range(b.shape[0]):
            difference[ix, qx] = (a[ix] - b[qx]) / bandwidth
            kernel_matrix[ix, qx] = difference[ix, qx]
            integrated_kernel_matrix[ix, qx] = difference[ix, qx]


    return difference, kernel_matrix, integrated_kernel_matrix

In [18]:
#@jit(nopython=True, cache=True, fastmath=True)
def difference_kernels_vectorized(a, b, bandwidth):
    # Use broadcasting to calculate the difference matrix
    difference = (a[:, np.newaxis] - b) / bandwidth
    # Calculate the kernel matrices
    kernel_matrix = difference
    integrated_kernel_matrix = difference


    return difference, kernel_matrix, integrated_kernel_matrix

In [19]:
def function1(a, b, bandwidth):
    return difference_kernels(a, b, bandwidth)
    

def function2(a, b, bandwidth):
    return difference_kernels_vectorized(a, b, bandwidth)

path = '/Users/JUSC/Documents/xgbsurv_benchmarking/implementation_testing/simulation_data'
def comparison(num_runs = 10, size=10000):

    hazard = log_hazard = np.random.normal(0, 1, size)
    df = pd.read_csv(path+'/survival_simulation_'+str(size)+'.csv')
    n_samples = df.shape[0]
    bandwidth = 1.30 * math.pow(n_samples, -0.2)
    df.sort_values(by='time', inplace=True)
    time = df.time.to_numpy()
    event = df.event.to_numpy().astype('bool')
    # Empty list to store the execution times
    function1_times = []
    function2_times = []

    # Loop to run each function and record the execution times
    for i in range(num_runs):
        start_time = t.time()
        function1(a=hazard, b=hazard[event], bandwidth=bandwidth)
        end_time = t.time()
        function1_times.append(end_time - start_time)

        start_time = t.time()
        function2(a=hazard, b=hazard[event], bandwidth=bandwidth)
        end_time = t.time()
        function2_times.append(end_time - start_time)

    # Calculate the mean and standard deviation of the execution times for each function
    function1_mean = sum(function1_times) / len(function1_times)
    function1_std = pd.Series(function1_times).std()
    function2_mean = sum(function2_times) / len(function2_times)
    function2_std = pd.Series(function2_times).std()

    # Create a Pandas dataframe to display the results
    df = pd.DataFrame({
        'Function': ['Difference Loop Numba (no erf)', 'Difference Vectorized (no erf)'],
        'Mean': [function1_mean, function2_mean],
        'Standard Deviation': [function1_std, function2_std],
        'Sample Size': [size, size],
        'Number Repetitions': [num_runs, num_runs]
    })
    return df

df_1000 = comparison(num_runs = 50, size=1000)
print(df_1000.to_latex(index=False))
df_1000.to_csv('/Users/JUSC/Documents/xgbsurv/experiments/implementation_testing/results/kernel_difference_no_erf_comparison_1000.csv', index=False)
df_1000

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

In [ ]:
df_10000 = comparison(num_runs = 50, size=10000)
print(df_10000.to_latex(index=False))
df_10000.to_csv('/Users/JUSC/Documents/xgbsurv/experiments/implementation_testing/results/kernel_difference_no_erf_comparison_10000.csv', index=False)
df_10000

\begin{tabular}{lrrrr}
\toprule
                      Function &     Mean &  Standard Deviation &  Sample Size &  Number Repetitions \\
\midrule
Difference Loop Numba (no erf) & 0.096040 &            0.003516 &        10000 &                  50 \\
Difference Vectorized (no erf) & 0.057099 &            0.002259 &        10000 &                  50 \\
\bottomrule
\end{tabular}



/var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_96162/296707172.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_10000.to_latex(index=False))


,Function,Mean,Standard Deviation,Sample Size,Number Repetitions
0,Difference Loop Numba (no erf),0.096040,0.003516,10000,50
1,Difference Vectorized (no erf),0.057099,0.002259,10000,50


## Comparison with Erf function

In [20]:
@jit(nopython=True, cache=True, fastmath=True)
def gaussian_integrated_kernel(x):
    return 0.5 * (1 + erfmath(x / SQRT_TWO))


@jit(nopython=True, cache=True, fastmath=True)
def gaussian_kernel(x):
    return PDF_PREFACTOR * exp(-0.5 * (x**2))

In [21]:
@jit(nopython=True, cache=True, fastmath=True)
def difference_kernels(a, b, bandwidth):
    difference: np.array = np.empty(shape=(a.shape[0], b.shape[0]))
    kernel_matrix: np.array = np.empty(shape=(a.shape[0], b.shape[0]))
    integrated_kernel_matrix: np.array = np.empty(
        shape=(a.shape[0], b.shape[0])
    )
    for ix in range(a.shape[0]):
        for qx in range(b.shape[0]):
            difference[ix, qx] = (a[ix] - b[qx]) / bandwidth
            kernel_matrix[ix, qx] = gaussian_kernel(difference[ix, qx])
            integrated_kernel_matrix[ix, qx] = gaussian_integrated_kernel(
                difference[ix, qx]
            )


    return difference, kernel_matrix, integrated_kernel_matrix

In [22]:
@njit
def erf_array(x):
    return np.array([math.erf(xi) for xi in x])

@njit
def erf_2d_array(x):
    m, n = x.shape
    result = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            result[i, j] = math.erf(x[i, j])
    return result


#@jit(nopython=True, cache=True, fastmath=True)
def gaussian_integrated_kernel_vec(x):
    #input =  x / SQRT_TWO
    return 0.5 * (1 + erfvec(x / SQRT_TWO))


#@jit(nopython=True, cache=True, fastmath=True)
def gaussian_kernel_vec(x):
    return PDF_PREFACTOR * np.exp(-0.5 * (x**2))

In [23]:
#@jit(nopython=True, cache=True, fastmath=True)
def difference_kernels_vectorized(a, b, bandwidth):
    # Use broadcasting to calculate the difference matrix
    difference = (a[:, np.newaxis] - b) / bandwidth
    # Calculate the kernel matrices
    kernel_matrix = gaussian_kernel_vec(difference)
    integrated_kernel_matrix = gaussian_integrated_kernel_vec(
                difference
            )

    return difference, kernel_matrix, integrated_kernel_matrix

In [24]:
## Compare times

In [25]:
def function1(a, b, bandwidth):
    return difference_kernels(a, b, bandwidth)
    

def function2(a, b, bandwidth):
    return difference_kernels_vectorized(a, b, bandwidth)

path = '/Users/JUSC/Documents/xgbsurv_benchmarking/implementation_testing/simulation_data'
def comparison(num_runs = 10, size=10000):

    hazard = log_hazard = np.random.normal(0, 1, size)
    df = pd.read_csv(path+'/survival_simulation_'+str(size)+'.csv')
    n_samples = df.shape[0]
    bandwidth = 1.30 * math.pow(n_samples, -0.2)
    df.sort_values(by='time', inplace=True)
    time = df.time.to_numpy()
    event = df.event.to_numpy().astype('bool')
    # Empty list to store the execution times
    function1_times = []
    function2_times = []

    # Loop to run each function and record the execution times
    for i in range(num_runs):
        start_time = t.time()
        function1(a=hazard, b=hazard[event], bandwidth=bandwidth)
        end_time = t.time()
        function1_times.append(end_time - start_time)

        start_time = t.time()
        function2(a=hazard, b=hazard[event], bandwidth=bandwidth)
        end_time = t.time()
        function2_times.append(end_time - start_time)

    # Calculate the mean and standard deviation of the execution times for each function
    function1_mean = sum(function1_times) / len(function1_times)
    function1_std = pd.Series(function1_times).std()
    function2_mean = sum(function2_times) / len(function2_times)
    function2_std = pd.Series(function2_times).std()

    # Create a Pandas dataframe to display the results
    df = pd.DataFrame({
        'Function': ['Difference Loop Numba', 'Difference Vectorized Numpy/Scipy'],
        'Mean': [function1_mean, function2_mean],
        'Standard Deviation': [function1_std, function2_std],
        'Sample Size': [size, size],
        'Number Repetitions': [num_runs, num_runs]
    })
    return df

df_1000 = comparison(num_runs = 50, size=1000)
print(df_1000.to_latex(index=False))
df_1000.to_csv('/Users/JUSC/Documents/xgbsurv/experiments/implementation_testing/results/kernel_difference_comparison_1000.csv', index=False)
df_1000

\begin{tabular}{lrrrr}
\toprule
Function & Mean & Standard Deviation & Sample Size & Number Repetitions \\
\midrule
Difference Loop Numba & 0.006490 & 0.000718 & 1000 & 50 \\
Difference Vectorized Numpy/Scipy & 0.005898 & 0.000380 & 1000 & 50 \\
\bottomrule
\end{tabular}



,Function,Mean,Standard Deviation,Sample Size,Number Repetitions
0,Difference Loop Numba,0.006490,0.000718,1000,50
1,Difference Vectorized Numpy/Scipy,0.005898,0.000380,1000,50


In [ ]:
df_1000.to_csv('/Users/JUSC/Documents/xgbsurv/experiments/implementation_testing/results/kernel_difference_comparison_10000.csv', index=False)

In [10]:
import numba
from numba import njit, jit
#import numbascipy
import scipy
from scipy.special import erf
import numpy as np

In [11]:
import numba; numba.njit(lambda x: x + 1)(123)

124

In [12]:
erf(2)

0.9953222650189527

In [13]:
@numba.njit
def compute_erf(a):
    return erf(a)

# Create an array
a = np.arange(10)

# Compute erf
result = compute_erf(a)
print(result)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<ufunc 'erf'>) found for signature:
 
 >>> erf(array(int64, 1d, C))
 
There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload of function 'erf': File: numba_scipy/special/overloads.py: Line 9.
        With argument(s): '(array(int64, 1d, C))':
       No match.

During: resolving callee type: Function(<ufunc 'erf'>)
During: typing of call at /var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_8566/3383678389.py (3)


File "../../../../../../var/folders/jr/dh6mkdzs31lc5pkqymtdbh180000gp/T/ipykernel_8566/3383678389.py", line 3:
<source missing, REPL/exec in use?>


In [44]:

def erf_array(x):
    return np.array([[math.erf(i) for i in row] for row in x])

@njit
def erf_2d_array(x):
    m, n = x.shape
    result = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            result[i, j] = math.erf(x[i, j])
    return result

from math import erf


#@jit(nopython=True, cache=True, fastmath=True)
def gaussian_integrated_kernel_vec(x):
    input =  x / SQRT_TWO
    return 0.5 * (1 +erf_array(input))


#@jit(nopython=True, cache=True, fastmath=True)
def gaussian_kernel_vec(x):
    return PDF_PREFACTOR * np.exp(-0.5 * (x**2))

#@jit(nopython=True, cache=True, fastmath=True)
def difference_kernels_vectorized(a, b, bandwidth):
    # Use broadcasting to calculate the difference matrix
    difference = (a[:, np.newaxis] - b) / bandwidth
    # Calculate the kernel matrices
    kernel_matrix = gaussian_kernel_vec(difference)
    integrated_kernel_matrix = gaussian_integrated_kernel_vec(
                difference
            )

    return difference, kernel_matrix, integrated_kernel_matrix

def function1(a, b, bandwidth):
    return difference_kernels(a, b, bandwidth)
    

def function2(a, b, bandwidth):
    return difference_kernels_vectorized(a, b, bandwidth)

path = '/Users/JUSC/Documents/xgbsurv_benchmarking/implementation_testing/simulation_data'
def comparison(num_runs = 10, size=10000):

    hazard = log_hazard = np.random.normal(0, 1, size)
    df = pd.read_csv(path+'/survival_simulation_'+str(size)+'.csv')
    n_samples = df.shape[0]
    bandwidth = 1.30 * math.pow(n_samples, -0.2)
    df.sort_values(by='time', inplace=True)
    time = df.time.to_numpy()
    event = df.event.to_numpy().astype('bool')
    # Empty list to store the execution times
    function1_times = []
    function2_times = []

    # Loop to run each function and record the execution times
    for i in range(num_runs):
        start_time = t.time()
        function1(a=hazard, b=hazard[event], bandwidth=bandwidth)
        end_time = t.time()
        function1_times.append(end_time - start_time)

        start_time = t.time()
        function2(a=hazard, b=hazard[event], bandwidth=bandwidth)
        end_time = t.time()
        function2_times.append(end_time - start_time)

    # Calculate the mean and standard deviation of the execution times for each function
    function1_mean = sum(function1_times) / len(function1_times)
    function1_std = pd.Series(function1_times).std()
    function2_mean = sum(function2_times) / len(function2_times)
    function2_std = pd.Series(function2_times).std()

    # Create a Pandas dataframe to display the results
    df = pd.DataFrame({
        'Function': ['Difference Loop Numba', 'Difference Vectorized Numpy/Scipy'],
        'Mean': [function1_mean, function2_mean],
        'Standard Deviation': [function1_std, function2_std],
        'Sample Size': [size, size],
        'Number Repetitions': [num_runs, num_runs]
    })
    return df

df_1000 = comparison(num_runs = 50, size=10000)
print(df_1000.to_latex(index=False))
df_1000.to_csv('/Users/JUSC/Documents/xgbsurv/experiments/implementation_testing/results/kernel_difference_comparison_1000.csv', index=False)
df_1000

\begin{tabular}{lrrrr}
\toprule
Function & Mean & Standard Deviation & Sample Size & Number Repetitions \\
\midrule
Difference Loop Numba & 0.732395 & 0.009505 & 10000 & 50 \\
Difference Vectorized Numpy/Scipy & 3.517251 & 0.055948 & 10000 & 50 \\
\bottomrule
\end{tabular}



,Function,Mean,Standard Deviation,Sample Size,Number Repetitions
0,Difference Loop Numba,0.732395,0.009505,10000,50
1,Difference Vectorized Numpy/Scipy,3.517251,0.055948,10000,50


[[0.8427007929497148, 0.9953222650189527, 0.9999779095030015, 0.0],
 [0.9953222650189527, 0.9953222650189527, 0.9953222650189527, 0.0],
 [0.9999779095030015, 0.9999779095030015, 0.9999779095030015, 0.0]]

In [14]:
from xgboost import XGBRegressor 
import numpy as np
x = np.random.normal(0, 1, size=100)
x = np.tile(x, (1,2))
y = np.random.normal(0, 1, size=100)

In [16]:
print(XGBRegressor().fit(x,y).__doc__)

Implementation of the scikit-learn API for XGBoost regression.


Parameters
----------

    n_estimators : int
        Number of gradient boosted trees.  Equivalent to number of boosting
        rounds.

    max_depth :  Optional[int]
        Maximum tree depth for base learners.
    max_leaves :
        Maximum number of leaves; 0 indicates no limit.
    max_bin :
        If using histogram-based algorithm, maximum number of bins per feature
    grow_policy :
        Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
        depth-wise. 1: favor splitting at nodes with highest loss change.
    learning_rate : Optional[float]
        Boosting learning rate (xgb's "eta")
    verbosity : Optional[int]
        The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
    objective : typing.Union[str, typing.Callable[[numpy.ndarray, numpy.ndarray], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]
        Specify the learning task and the correspon

In [13]:
x

array([], shape=(0, 200), dtype=float64)

In [61]:
n_events

array([3., 1., 2., 1.])

In [54]:
event_torch.shape

torch.Size([8])